

### Image segementaion model train 2022 Nov




## 1. Import packages

In [ ]:
!pip install torchvision --upgrade
!pip install grad-cam
!pip install timm
!pip install imagecodecs
!pip install git+https://github.com/qubvel/segmentation_models.pytorch

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
from torchvision.datasets import VisionDataset
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader, Subset

import os 
from os import path

import numpy as np
import pandas as pd
from scipy.io import loadmat

from tqdm import tqdm
from PIL import Image

# read tiff
import zipfile
from tifffile import imread
from torchvision.transforms import ToTensor
import random
import csv
import matplotlib.pyplot as plt
import cv2 as cv

In [ ]:
from unet import UNet
from keyholeDataset import Keyhole
from augmentation import get_training_augmentation
from loss import DiceBCELoss
from utils import plot_2_sidebyside, plot_3_sidebyside, save_model
from train import train
from validation import validation
# import segmentation_models_pytorch as smp


## 2. Initiate a model

In [ ]:
#model = torch.hub.load('milesial/Pytorch-UNet', 'unet_carvana', pretrained=True, scale=0.5)

model = UNet(n_channels=3, n_classes=1, bilinear=1)

torch.cuda.empty_cache()
model.cuda()

In [ ]:
# # load model
# checkpoint = torch.load("unet_test_epoch_51")
# model.load_state_dict(checkpoint['model_state_dict'])

## 3. load data + specify batch_size and epochs

In [ ]:
!mkdir Keyhole

from google.colab import drive
drive.mount('/content/drive')

with zipfile.ZipFile('/content/drive/MyDrive/DL_segmentation_data/keyhole_segmentation_data.zip', 'r') as zip:
  zip.extractall(path='/content/Keyhole')


cuda = torch.cuda.is_available()
device = torch.device("cuda" if cuda else "cpu")
num_workers = 4 if cuda else 0
print("Cuda = " + str(cuda)+" with num_workers = "+str(num_workers))


In [ ]:
# need to write config file to make this part elegent
batch_size = 1
epochs = 300

train_dataset = Keyhole('/content/Keyhole/keyhole_segmentation_data', transform=get_training_augmentation(), mode="train", csv_name="/image_and_split_1.csv")
val_dataset = Keyhole('/content/Keyhole/keyhole_segmentation_data', transform=None , mode="val", csv_name="/image_and_split_1.csv")
test_dataset = Keyhole('/content/Keyhole/keyhole_segmentation_data', transform=None, mode="test", csv_name="/image_and_split_1.csv")

print(f"Train size: {len(train_dataset)}")
print(f"Valid size: {len(val_dataset)}")
print(f"Test size: {len(test_dataset)}")


train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=1)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=1)

In [ ]:
pred_masks = []
for i, batch in enumerate(train_loader):
      print("i = ", i)
      x = batch['image'].float().to(device) 
      y = batch['mask'].float().to(device) 
      assert(len(x) == len(y))
      #print("x shape", x.shape) #torch.Size([1, 3, 572, 572])
      #print("y shape", y.shape) #torch.Size([1, 1, 572, 572])
      # yp = model(x)
      # print("yp shape", yp.shape)#torch.Size([1, 1, 572, 572])
      plot_2_sidebyside(x.detach().cpu().numpy()[0][0], 
                      y.detach().cpu().numpy()[0][0])
                      
      
      # plot_3_sidebyside(x.detach().cpu().numpy()[0][0], 
      #                 y.detach().cpu().numpy()[0][0], 
      #                 yp.detach().cpu().numpy()[0][0]), (yp.detach().cpu().numpy()[0][0]>0.5).astype(int))


## 4. Model training

In [ ]:
# #del model
# torch.cuda.empty_cache()
# model.cuda()

In [ ]:
from torchsummary import summary
summary(model, (3, 572, 572))

In [ ]:
 # 4. Set up the optimizer, the loss, the learning rate scheduler and the loss scaling for AMP
 # https://github.com/milesial/Pytorch-UNet/blob/master/train.py
optimizer = optim.RMSprop(model.parameters(), lr=1e-5, weight_decay=1e-8, momentum=0.99)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=2)  # goal: maximize Dice score
grad_scaler = torch.cuda.amp.GradScaler(enabled=True)
criterion = nn.BCEWithLogitsLoss()

In [ ]:
epochs = 50
amp = True
train_losses= []
val_losses= []
for epoch in range(1, epochs+1):
  train_loss = train(model, device, train_loader, optimizer, criterion, scheduler, grad_scaler, epoch, epochs=300, amp=True)
  train_losses.append(train_loss)
  val_loss = validation(model, device, val_loader, optimizer, criterion, scheduler, epoch, epochs=300, amp=True)
  val_losses.append(val_loss)
  if epoch in {50, 100, 150, 200, 250, 300}:
    save_model(model, epoch, "unet", optimizer, scheduler, grad_scaler, batch_size)

## 5. Save model and plot loss

In [ ]:
plt.figure(figsize=(10,10))
plt.plot(loss_rec)

In [ ]:
save_model(model, 50, "unet_test", optimizer, scheduler, 1)

In [ ]:
checkpoint = torch.load("unet_test_epoch_51")
model.load_state_dict(checkpoint['model_state_dict'])